# Summary

In [1]:
from abstractor.train import get_training_batch as get_abstractor_training_batch
from abstractor.utils import AbstractorModel, AbstractorModelRNN
from abstractor.utils import obtain_initial_hidden_states
from bert.utils import obtain_sentence_embeddings
from bert.utils import obtain_word_embeddings
from data.utils import load_training_dictionaries
from extractor.train import get_training_batch as get_extractor_training_batch
from extractor.utils import ExtractorModel
from pytorch_transformers import BertModel
from pytorch_transformers import BertTokenizer
from rl_connection.utils import RLModel
from rl_connection.train import get_training_batch as get_rl_training_batch
from rouge import Rouge

import numpy as np
import torch

## Load data

In [2]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

In [3]:
data = load_training_dictionaries()

## Extractor

In [4]:
# Load extractor model:
extractor_model = ExtractorModel(bert_tokenizer, bert_model)
extractor_model_path = "results/models/extractor.pt"
extractor_model.load_state_dict(torch.load(extractor_model_path))

<All keys matched successfully>

In [5]:
documents, extraction_labels = get_extractor_training_batch(data, batch_size=2)

sentence_embeddings, mask = obtain_sentence_embeddings(
    extractor_model.bert_model, 
    extractor_model.bert_tokenizer, 
    documents
)

# Predict probability of extraction per sentence
extraction_probabilities = extractor_model(sentence_embeddings)

In [6]:
n_samples = len(documents)

for sample_idx in range(n_samples):
    n_to_extract = extraction_labels.sum(dim=1)[sample_idx].int() 
    ext_prob = extraction_probabilities[sample_idx] * mask[sample_idx]
    ext_sent_indicies = torch.topk(ext_prob, k=n_to_extract)[1]
    
    targets = np.array(documents[sample_idx])[extraction_labels[sample_idx][:len(documents[sample_idx])].numpy().astype(bool)]
    print("----> TARGET <----")
    for target in targets:
        print(f"{target}\n")
    print()
          
    print("----> PREDICTION <----")
    for x in np.array(documents[sample_idx])[ext_sent_indicies]:
        print(f"{x} \n")
    print("\n\n-------\n\n")

----> TARGET <----
but when rory mcilroy pulled his second shot on the eighth hole of the wgc cadillac championship into a lake friday , he might as well have been channeling the much loved adam sandler character .

before continuing his round with a dropped ball , the four - time major winner launched the 3 - iron used to play the offending shot into the water as well .


----> PREDICTION <----
but when rory mcilroy pulled his second shot on the eighth hole of the wgc cadillac championship into a lake friday , he might as well have been channeling the much loved adam sandler character . 

before continuing his round with a dropped ball , the four - time major winner launched the 3 - iron used to play the offending shot into the water as well . 



-------


----> TARGET <----
( cnn ) french striker bafetimbi gomis , who has a history of fainting , said he is now " feeling well " after collapsing during swansea 's 3 - 2 loss at tottenham in the premier league on wednesday .

the worryi

## Abstractor

In [7]:
# Load data:
abstractor_model = AbstractorModelRNN(bert_tokenizer, bert_model)
abstractor_model_path = "results/models/abstractor.pt"
abstractor_model.load_state_dict(torch.load(abstractor_model_path))

<All keys matched successfully>

In [8]:
source_documents, target_summaries = get_abstractor_training_batch(data, 1)

# Obtain embeddings
source_document_embeddings, source_mask, source_tokens = obtain_word_embeddings(
    abstractor_model.bert_model, abstractor_model.bert_tokenizer, source_documents, static_embeddings=False
)
target_summary_embeddings, target_mask, target_tokens = obtain_word_embeddings(
    abstractor_model.bert_model, abstractor_model.bert_tokenizer, target_summaries, static_embeddings=True
)

print(source_documents)
print()
print(target_summaries)

[['( cnn ) french striker bafetimbi gomis , who has a history of fainting , said he is now " feeling well " after collapsing during swansea \'s 3 - 2 loss at tottenham in the premier league on wednesday .', 'the worrying incident occurred in the first half at white hart lane -- after tottenham scored in the seventh minute -- but the 29 - year - old left the pitch conscious following about five minutes of treatment . the guardian added that he was wearing an oxygen mask .', '" i wanted to reassure you concerning my health , " gomis told the website . " it actually looks much scarier than it is physically dangerous , and i am feeling well now .', 'almost exactly three years ago at white hart lane , then bolton midfielder fabrice muamba collapsed after suffering a cardiac arrest . he was near death , according to bolton , but survived after being treated at the london chest hospital .']]

[['bafetimbi gomis collapses within 10 minutes of kickoff at tottenham .', 'but he reportedly left th

In [9]:
# Obtain extraction probability for each word in vocabulary
extraction_probabilities, teacher_forcing = abstractor_model(
    source_document_embeddings,
    target_summary_embeddings,
    teacher_forcing_pct=0
)  # (batch_size, n_target_words, vocab_size)

vals, predicted_idx = torch.topk((extraction_probabilities), k=1, dim=2)

for x in [abstractor_model.bert_tokenizer.convert_ids_to_tokens(p) for p in predicted_idx.squeeze().tolist()]:
    print(f"{x}")
    print("")
    
print("------------")


# Obtain extraction probability for each word in vocabulary
extraction_probabilities, teacher_forcing = abstractor_model(
    source_document_embeddings,
    target_summary_embeddings,
    teacher_forcing_pct=1
)  # (batch_size, n_target_words, vocab_size)

vals, predicted_idx = torch.topk((extraction_probabilities), k=1, dim=2)

for x in [abstractor_model.bert_tokenizer.convert_ids_to_tokens(p) for p in predicted_idx.squeeze().tolist()]:
    print(f"{x}")
    print("")

ba

##fe

##ti

##mb

##i

collapses

10

minutes

tottenham

the

pitch

conscious

and

wearing

wearing

wearing

wearing

wearing

wearing

wearing

wearing

wearing

wearing

wearing

wearing

wearing

wearing

wearing

wearing

wearing

wearing

wearing

and

wearing

wearing

wearing

wearing

wearing

wearing

wearing

wearing

wearing

wearing

wearing

wearing

wearing

wearing

wearing

wearing

wearing

wearing

wearing

wearing

wearing

wearing

wearing

wearing

wearing

wearing

------------
ba

##fe

##ti

##mb

##i

collapses

##mis

collapses

10

10

minutes

tottenham

10

##mis

tottenham

the

but

the

reportedly

left

he

pitch

conscious

and

wearing

wearing

oxygen

mask

said

go

said

later

said

that

.

"

"

feeling

well

incident

feeling

incident

after

three

years

after

fabric

##e

mu

##am

##ba

collapsed

.

white

hart

white

.

[SEP]

at



## Reinforcement Learning

In [61]:
rl_model = RLModel(extractor_model, abstractor_model)
rl_model.load_state_dict(torch.load("results/models/rl.pt"))

<All keys matched successfully>

In [64]:
source_documents, target_summaries = get_rl_training_batch(data, batch_size=1)

# Obtain embeddings
source_sentence_embeddings, source_mask = obtain_sentence_embeddings(
    rl_model.extractor_model.bert_model,
    rl_model.extractor_model.bert_tokenizer,
    source_documents
)
stop_action_index = source_sentence_embeddings.shape[1]
target_summary_embeddings, target_mask, target_tokens = obtain_word_embeddings(
    rl_model.abstractor_model.bert_model,
    rl_model.abstractor_model.bert_tokenizer,
    target_summaries,
    static_embeddings=True
)

# Run trajectory
actions, log_probs, entropys, values, n_ext_sents = rl_model.sample_actions(
    source_sentence_embeddings,
    source_mask
)

# Obtain abstracted sentence from abstractor
predicted_tokens, word_probabilities = rl_model.create_abstracted_sentences(
    actions,
    source_documents,
    n_ext_sents=n_ext_sents,
    teacher_forcing_pct=0,
    target_summary_embeddings=target_summary_embeddings
)

# Look at extractions
for art_idx, doc_sentences in enumerate(actions):
    for sent_idx in doc_sentences[:-1]:
        print(source_documents[art_idx][sent_idx])
        print()
    print("\n\n-------\n\n")
    
# Look at abstractions
for predicted_abstraction in predicted_tokens:
    solution = list()
    for token in predicted_abstraction:
        solution.append(rl_model.abstractor_model.bert_tokenizer.ids_to_tokens[int(token)])
    print(" ".join(solution))
    print("\n\n")
    
print("-------\n\n")
target_summaries

that may sound like an esoteric adage , but when zully broussard selflessly decided to give one of her kidneys to a stranger , her generosity paired up with big data . it resulted in six patients receiving transplants .

that changed when a computer programmer named david jacobs received a kidney transplant . he had been waiting on a deceased donor list , when a live donor came along -- someone nice enough to give away a kidney to a stranger .

here 's how the super swap works , according to california pacific medical center .



-------


zu ##lly bro ##uss ##ard decided bro ##uss ##ard decided computer ##uss ##ard decided computer ##uss ##ard kidney computer ##uss ##ard kidney computer give give ##ard kidney computer give give ##ard



-------




[['zully broussard decided to give a kidney to a stranger .',
  'a new computer program helped her donation spur transplants for six kidney patients .']]

In [123]:
a = torch.tensor([
    [1, 2, 3],
    [1, 2, 3],
    [1, 2, 3],
])
a = a.unsqueeze(1)

b = torch.tensor([
    [10, 10, 10],
    [20, 20, 20],
])


In [124]:
print(a.shape)
print(b.shape)

torch.Size([3, 1, 3])
torch.Size([2, 3])


In [127]:
c = a + b
print(c)
c.shape

tensor([[[11, 12, 13],
         [21, 22, 23]],

        [[11, 12, 13],
         [21, 22, 23]],

        [[11, 12, 13],
         [21, 22, 23]]])


torch.Size([3, 2, 3])

In [88]:
from collections import defaultdict

label_indicies_dict = defaultdict(list)
for row, pos in zip(b, c):
    label_indicies_dict[row.item()].append(pos.item())

In [89]:
label_indicies_dict

defaultdict(list, {0: [0], 1: [1, 2], 2: [0, 1]})

In [95]:
a[0][[0, 0, 0]]

tensor([1, 1, 1])

In [141]:
a = torch.rand(7, 3)
b = torch.rand(4, 3)

In [142]:
print(a)
print()
print(b)

tensor([[0.4570, 0.0248, 0.8170],
        [0.7475, 0.7047, 0.1016],
        [0.4365, 0.9295, 0.2564],
        [0.7821, 0.1784, 0.2322],
        [0.6998, 0.4841, 0.0119],
        [0.6714, 0.4237, 0.2638],
        [0.8909, 0.6114, 0.3419]])

tensor([[0.7274, 0.1380, 0.3910],
        [0.6914, 0.3259, 0.6325],
        [0.6592, 0.5140, 0.6638],
        [0.7398, 0.5345, 0.7570]])


In [143]:
c = a.unsqueeze(1) + b
print(c)
c.shape

tensor([[[1.1843, 0.1628, 1.2079],
         [1.1484, 0.3506, 1.4494],
         [1.1162, 0.5387, 1.4807],
         [1.1967, 0.5593, 1.5740]],

        [[1.4748, 0.8427, 0.4925],
         [1.4389, 1.0306, 0.7340],
         [1.4067, 1.2187, 0.7653],
         [1.4873, 1.2392, 0.8586]],

        [[1.1638, 1.0675, 0.6474],
         [1.1279, 1.2554, 0.8889],
         [1.0957, 1.4435, 0.9202],
         [1.1762, 1.4641, 1.0135]],

        [[1.5094, 0.3164, 0.6232],
         [1.4735, 0.5042, 0.8647],
         [1.4413, 0.6923, 0.8960],
         [1.5218, 0.7129, 0.9892]],

        [[1.4271, 0.6221, 0.4029],
         [1.3912, 0.8099, 0.6444],
         [1.3590, 0.9980, 0.6757],
         [1.4396, 1.0186, 0.7690]],

        [[1.3987, 0.5617, 0.6548],
         [1.3628, 0.7496, 0.8963],
         [1.3306, 0.9377, 0.9276],
         [1.4111, 0.9582, 1.0209]],

        [[1.6183, 0.7494, 0.7329],
         [1.5823, 0.9373, 0.9744],
         [1.5502, 1.1254, 1.0057],
         [1.6307, 1.1459, 1.0990]]])


torch.Size([7, 4, 3])

In [139]:
a[0] + b

tensor([[1.5932, 1.2935, 0.9342],
        [0.9397, 0.9672, 1.2095],
        [1.1525, 0.9470, 1.1873],
        [1.6448, 1.3081, 0.5084]])